In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from track import get_infected_img
from track import get_tracks, plot_tracks, get_events_df
from tqdm import tqdm
import sys
from pathlib import Path
sys.path.insert(0, '..') # in order to be able to import from scripts.py
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [ ]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

# Tracking simulations

In [ ]:
from laptrack import LapTrack

def get_laptrack(max_distance = 25):
    lt = LapTrack(
        track_dist_metric="sqeuclidean",  
        splitting_dist_metric="sqeuclidean",
        merging_dist_metric="sqeuclidean",
        track_cost_cutoff=max_distance**2,
        splitting_cost_cutoff=max_distance**2,  # False for non-splitting case
        merging_cost_cutoff=False,  # False for non-merging case
        )
    return lt

def cut_small_trees(track_df, size = 10):
    tree_count = track_df.groupby('tree_id').size()
    false_trees = tree_count[tree_count < size].index.values
    track_df = track_df[~track_df.tree_id.isin(false_trees)]
    for index in false_trees:
        track_df.loc[track_df.tree_id > index, 'tree_id'] -= 1
    return track_df

In [ ]:
roll = 20
std_factor = 1/10
max_distance = 30
sim_num = 500
folder = "results/data150"

events = []
too_many_splits = []
lt = get_laptrack(max_distance=max_distance)
for i in tqdm(range(sim_num)):
    df = pd.read_csv(f"{folder}/out-{i}.csv")
    img = get_infected_img(df)
    track_df, split_df, merge_df = get_tracks(img, lt, roll=roll, std_factor=std_factor)

    track_df = cut_small_trees(track_df)
    
    if (track_df['tree_id'] > 1).any():
        too_many_splits.append(i)
    if len(split_df) == 0:
        events.append(pd.DataFrame())
        continue
    events_df = get_events_df(track_df, split_df)
    events.append(events_df)

In [ ]:
df_results = pd.concat(events, names=['sim_id'], keys=range(sim_num))
too_many_splits

In [ ]:
df_results['tree_id'].value_counts()

## Testing

In [ ]:
def debug(sim_path, max_distance, roll = 20, std_factor = 1/10):
    df = pd.read_csv(sim_path)
    img = get_infected_img(df)
    lt = get_laptrack(max_distance=max_distance)
    track_df, split_df, merge_df = get_tracks(img, lt, roll=roll, std_factor=std_factor)
    
    plot_tracks(track_df, split_df, merge_df)
    return track_df, split_df

In [ ]:
for i, sim_id in enumerate(too_many_splits):
    sim_path = f"results/data150/out-{sim_id}.csv"
    track_df, _ = debug(sim_path, max_distance=30, roll = 20, std_factor=1/10)
    plt.title(f"Simulation id: {sim_id}, {len(track_df[track_df['tree_id'] > 1])}")
    plt.show()


In [ ]:
first_signal_splits = df_results[df_results['tree_id'] == 1].groupby('sim_id').size().sort_values(ascending=False).index.values
for i, sim_id in enumerate(first_signal_splits[:10]):
    sim_path = f"results/data150/out-{sim_id}.csv"
    plt.title(f"Simulation id: {sim_id}")
    debug(sim_path, max_distance=30, roll=20, std_factor=1/10)
    plt.show()

# Get data

In [ ]:
sim_num =500
folder = "results/data150"
Path(folder).mkdir(exist_ok=True, parents=True)

In [ ]:
for channel_height in [7]:

    client = VisAVisClient(
        visavis_bin=f'../target/bins/vis-a-vis-{channel_height}',
    )

    for interval in [150]:

        protocol_file_path = make_protocol(
            pulse_intervals = [interval, 1500],
            duration=4,
            out_folder='./'
        )
        
        for sim in tqdm(range(sim_num)):
            result = client.run(
                parameters_json=PARAMETERS_DEFAULT,
                protocol_file_path= protocol_file_path,
            )
            result.states.to_csv(f"{folder}/out-{sim}.csv", index=False)